In [ ]:
#Daily tester

In [ ]:
import sys, subprocess, pathlib

root = pathlib.Path("/Users/keanupanapa/Notebooks/Forex_Backtester")
script = root / "smoke_test_selfcontained_v198.py"

cmd = [sys.executable, str(script), "--mode", "fast"]  # or "full"
print("Running:", " ".join(cmd))
proc = subprocess.run(cmd, capture_output=True, text=True)
print(proc.stdout)
print(proc.stderr)
if proc.returncode != 0:
    raise RuntimeError(f"Self-contained smoke failed with code {proc.returncode}")


In [4]:
# --- Audit critical hooks inside backtester.py (non-destructive) ---

import importlib, inspect, re, pathlib, sys

project_root = pathlib.Path.cwd()
mod_name = "backtester"

# (Re)load backtester freshly
if mod_name in sys.modules:
    del sys.modules[mod_name]
bt = importlib.import_module(mod_name)

report = {}

def has_func(name, min_params=None):
    ok = hasattr(bt, name) and callable(getattr(bt, name))
    if ok and min_params is not None:
        sig = inspect.signature(getattr(bt, name))
        ok = len(sig.parameters) >= min_params
    return ok

def find_source_snippet(pattern):
    try:
        src = inspect.getsource(bt)
    except Exception:
        return False, None
    m = re.search(pattern, src, flags=re.DOTALL)
    return (m is not None), (src[m.start():m.end()] if m else None)

# 1) WFO entrypoint(s)
report["WFO.run_backtest_walk_forward"] = has_func("run_backtest_walk_forward", min_params=1)

# 2) cache-aware indicator application
report["apply_indicators_with_cache"] = has_func("apply_indicators_with_cache")

# 3) DBCVIX canonical risk resolver (3-tuple)
report["resolve_dbcvix_risk"] = has_func("resolve_dbcvix_risk", min_params=4)

# 4) DBCVIX series loader wired at run_backtest()
report["load_dbcvix_series"] = has_func("load_dbcvix_series")
src = inspect.getsource(bt.run_backtest)
report["run_backtest_uses_load_dbcvix_series"] = "load_dbcvix_series" in src

# 5) trailing-stop fill/audit fix block present?
ts_fix_found, _ = find_source_snippet(r'reason\s*==\s*["\']trailing_stop["\']\s*:\s*\n\s*#?\s*Force fill at the final stop')
report["trailing_stop_exit_fill_fix"] = ts_fix_found

# 6) trades schema includes DBCVIX audit fields
try:
    TRADES_COLS = getattr(bt, "TRADES_COLS", [])
except Exception:
    TRADES_COLS = []
need_cols = {"risk_pct_used", "dbcvix_val", "dbcvix_flag"}
report["TRADES_COLS_has_dbcvix_fields"] = bool(set(TRADES_COLS) >= need_cols)

# 7) simulate_pair_trades signature + returns equity when asked
report["simulate_pair_trades"] = has_func("simulate_pair_trades", min_params=4)
sim_src = inspect.getsource(bt.simulate_pair_trades)
report["simulate_pair_trades_return_equity_flag"] = ("return_equity" in sim_src)

# 8) finalize stop persisted on exit
persist_fix_found, _ = find_source_snippet(r'finalize_trade_row\([^)]*current_stop_price_at_exit')
report["persist_stop_at_exit"] = persist_fix_found

# 9) spreads modeled in PnL-only via spread_pips_used
spread_fix_found, _ = find_source_snippet(r'compute_trade_pnl_money\(.+?spread_pips_used',)
report["spread_in_pnl_model"] = spread_fix_found

print("=== backtester audit ===")
for k,v in report.items():
    print(f"{k:38} -> {v}")


=== backtester audit ===
WFO.run_backtest_walk_forward          -> True
apply_indicators_with_cache            -> True
resolve_dbcvix_risk                    -> True
load_dbcvix_series                     -> True
run_backtest_uses_load_dbcvix_series   -> True
trailing_stop_exit_fill_fix            -> False
TRADES_COLS_has_dbcvix_fields          -> True
simulate_pair_trades                   -> True
simulate_pair_trades_return_equity_flag -> True
persist_stop_at_exit                   -> False
spread_in_pnl_model                    -> True
